加载云端硬盘，将Google云盘的压缩包在项目空间中解压，参考：https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook

曾用：`!unzip "/content/drive/MyDrive/Colab Notebooks/dataset.zip" -d "/content/"`

In [1]:
!unzip "/content/drive/MyDrive/Datasets/dataset_day.zip" -d "/content/"

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/dataset/labels/train/9fa89440-4bba242f.txt  
  inflating: /content/dataset/labels/train/9fa97d4c-2ff831e6.txt  
  inflating: /content/dataset/labels/train/9fabb561-3ca93cc2.txt  
  inflating: /content/dataset/labels/train/9faf65e3-443c77eb.txt  
  inflating: /content/dataset/labels/train/9faf65e3-c75ed114.txt  
  inflating: /content/dataset/labels/train/9fb0662d-a8057e2e.txt  
  inflating: /content/dataset/labels/train/9fb1582d-7326109f.txt  
  inflating: /content/dataset/labels/train/9fb1ab32-15504950.txt  
  inflating: /content/dataset/labels/train/9fb1b785-1236af2d.txt  
  inflating: /content/dataset/labels/train/9fb1b785-3c16c3d3.txt  
  inflating: /content/dataset/labels/train/9fb1d51e-3dcd0c0d.txt  
  inflating: /content/dataset/labels/train/9fb26f8c-2c2f002f.txt  
  inflating: /content/dataset/labels/train/9fb55d2c-2bf47cf7.txt  
  inflating: /content/dataset/labels/train/9fb57992-7d30ce84.txt  
  inflating: /content/dataset/label

安装ultralytics（YOLOv8）

In [2]:
#安装（已完成）
!pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 7.5 MB/s eta 0:00:00


In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.7/78.2 GB disk)


加载数据yaml（后续步骤参考：https://github.com/lyhue1991/eat_yolov8/blob/main/02_YOLOv8%E8%AE%AD%E7%BB%83%E8%87%AA%E5%B7%B1%E7%9A%84%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb ，并省略一些验证步骤）

In [3]:
%%writefile car.yaml
# Ultralytics YOLO 🚀, GPL-3.0 license

path: /content/dataset   # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test: image/test # test images (optional)

# Classes
names:
  0: car
  1: bus
  2: truck

Writing car.yaml


In [4]:
import torch
from torch.utils.data import DataLoader
# from ultralytics.yolo.cfg import get_cfg
# from ultralytics.yolo.utils import DEFAULT_CFG,yaml_load
# from ultralytics.yolo.data.utils import check_cls_dataset, check_det_dataset
# from ultralytics.yolo.data import build_yolo_dataset,build_dataloader
from ultralytics.cfg import get_cfg
from ultralytics.utils import DEFAULT_CFG, yaml_load
from ultralytics.data.utils import check_cls_dataset, check_det_dataset
from ultralytics.data import build_yolo_dataset, build_dataloader

overrides = {'task':'detect',
             'data':'car.yaml',
             'imgsz':(1280,720),
             'workers':4
            }
cfg = get_cfg(cfg = DEFAULT_CFG,overrides=overrides)
data_info = check_det_dataset(cfg.data)

100%|██████████| 755k/755k [00:00<00:00, 24.4MB/s]


省略中间步骤，进入定义模型阶段（~~第一次先从云端下载，后续再直接使用~~ 直接从本地导入，更加方便）

In [6]:
!cp "/content/drive/MyDrive/Colab Notebooks/yolov8n.pt" "/content/yolov8n.pt"

In [5]:
from ultralytics.nn.tasks import DetectionModel

model = DetectionModel(cfg = 'yolov8n.yaml', ch=3) # nc = 1? 3?
# weights = torch.hub.load_state_dict_from_url('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt')
# weights = torch.load('yolov8n.pt')
weights = torch.load('last.pt')
model.load(weights['model'])


model.args = cfg
model.nc = data_info['nc']  # attach number of classes to model
model.names = data_info['names']


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [8]:
model.args = cfg
model.nc = data_info['nc']  # attach number of classes to model
model.names = data_info['names']

训练模型阶段（初尝试：只跑100张图片，按8:1:1的比例分配train、val、和test；直接使用原生接口检测是否能跑通）

In [7]:
from ultralytics import YOLO
# yolo_model = YOLO('yolov8n.pt')
yolo_model = YOLO('last.pt')
yolo_model.train(data='car.yaml',epochs=20, resume=True)

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=last.pt, data=car.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=tor

train: Scanning /content/dataset/labels/train... 33589 images, 205 backgrounds, 0 corrupt: 100%|██████████| 33589/33589 [00:28<00:00, 1165.35it/s]

train: WARNING ⚠️ /content/dataset/images/train/75055858-7d04a650.jpg: 1 duplicate labels removed


train: New cache created: /content/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/dataset/labels/val... 4199 images, 25 backgrounds, 0 corrupt: 100%|██████████| 4199/4199 [00:03<00:00, 1325.42it/s]

...

      10/10      2.49G      1.066     0.6117     0.9427         65        640: 100%|██████████| 2100/2100 [15:21<00:00,  2.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 132/132 [00:59<00:00,  2.22it/s]


                   all       4199      49419      0.688      0.515      0.583      0.403

10 epochs completed in 2.758 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 132/132 [01:13<00:00,  1.79it/s]


                   all       4199      49419      0.688      0.515      0.583      0.404
                   car       4199      45885      0.781      0.672      0.734       0.47
                   bus       4199        952      0.644      0.412      0.485      0.366
                 truck       4199       2582      0.639       0.46      0.529      0.374
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a318a7ccb20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

从云端批量下载测试的结果（参考：[colab 从云端批量下载文件到本地\_colab 批量下载文件-CSDN博客](https://blog.csdn.net/jizhidexiaoming/article/details/108302794)）

In [9]:
import os, tarfile

import os
from google.colab import files

#逐个添加文件打包，未打包空子目录。可过滤文件。
#如果只打包不压缩，将"w:gz"参数改为"w:"或"w"即可。
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()

  files.download(output_filename)

make_targz_one_by_one('train_day.tar', '/content/runs/detect/train2')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

评估部分（直接使用yolo默认的接口，不需要了）